<a href="https://colab.research.google.com/github/Labibbib/MSA_Case_Study_1_Group-F/blob/main/CS_1_Group_F_preprocsess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Analisis Sentimen komentar dan Tanggapan Publik di youtube terhadap diluncurkannya Danantara**

---

**Latar Belakang**

Peluncuran Badan Pengelola Investasi Daya Anagata Nusantara (Danantara) oleh Presiden Prabowo Subianto pada 24 Februari 2025 menjadi langkah strategis dalam mengoptimalkan investasi negara. Inisiatif ini dipandang sebagai bagian dari reformasi BUMN dan didorong untuk mempercepat pertumbuhan ekonomi Indonesia hingga 8% per tahun. Meski dianggap sebagai langkah strategis, kehadiran Danantara juga menimbulkan kontroversi, terutama terkait transparansi, potensi korupsi, serta keterbatasan audit oleh BPK dan KPK.

---

**Mengapa penting untuk di analisis?**

Program ini menuai kontroversi karena dianggap sebagai “ladang korupsi baru”, sehingga perlu dilakukan analisis bagaimana sebenarnya pendapat masyarakat.
Hasil analisis dapat memberikan rekomendasi kepada pemerintah untuk memperbaiki kebijakan dan meningkatkan kepercayaan masyarakat.

---

**Group F - Case Study 1**

| Nama | NIM | Kelas | Kontribusi |
| -------- | -------- | -------- | -------- |
| Ibnullabib   | 11220940000037   |  A  |  |
| Uswatun Hasanah | 11220940000053   |   B |  |
| Muhammad Saeful Anwar | 11210940000061   |  B  |  |
| Dina Sekar Juliati | 11210940000020   |  A  |  |


# Import package yang dibutuhkan

In [1]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 8.7 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import matplotlib.pyplot as plt
import requests
import string


from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from wordcloud import WordCloud
from collections import Counter

# Load data

In [4]:
df = pd.read_excel('https://github.com/Labibbib/MSA_Case_Study_1_Group-F/raw/refs/heads/main/data_danantara/labeling_data%20youtube%20danantara.xlsx')
df = df.rename(columns={'text':'full_text'}, inplace=False)
df.sample(10)

,author,updated_at,like_count,full_text,video_id,public,label
7929,@suparta8303,2025-03-26T16:19:40Z,1,"Kerenn bang, sering2 bikin podcast",LTrJJ4X3D5Q,True,1.0
8739,@keneketodaily,2025-03-23T15:34:40Z,0,"6il4 loe ncek,,,, bang sandi belajar sama samp...",LTrJJ4X3D5Q,True,0.0
7175,@piurnams,2025-02-24T12:38:59Z,0,"wajib hukumnya program ini berhasil, kalo tida...",PNFTdgS0Mgc,True,1.0
519,@irzan007,2024-11-15T00:19:27Z,0,sama aja dengan jual BUMN,g3u2XDuQAUw,True,0.0
4661,@QonitaAkbar-j9t,2025-02-20T00:10:26Z,0,boleh aja tapi kalau pengelola nya terbukti ko...,02_WTfYvQiE,True,2.0
3178,@ekostv,2025-02-21T11:16:39Z,0,"ok bang, saya faham masalah ini, tapi kita den...",02_WTfYvQiE,True,0.0
6459,@ddiver2200,2025-02-28T11:12:46Z,0,"gak jauh beda dengan pemerintahan pendahulu, m...",WyPLvwLK8c4,True,0.0
9362,@Dimensity1200,2025-02-20T03:32:53Z,2,"Msh ingat kata² "" tak bekuasa selama 20 tahun ...",ruqEjXiJZGw,True,0.0
2778,@axellearzikial-zamahmadsya7988,2025-02-23T01:31:54Z,0,Masalahnya Dana yang dikelola oleh pemerintah ...,02_WTfYvQiE,True,0.0
7779,@KISWANTO_PODOKONO,2025-04-07T04:45:52Z,0,di beberapa bulan lalu sya bilang pd teman dan...,LTrJJ4X3D5Q,True,2.0


# Preprocessing

In [5]:
# Mengecek data yang null/kosong
print(df.isnull().sum())
df = df.dropna()

author        4
updated_at    0
like_count    0
full_text     0
video_id      0
public        0
label         8
dtype: int64


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9835 entries, 0 to 9846
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   author      9835 non-null   object 
 1   updated_at  9835 non-null   object 
 2   like_count  9835 non-null   int64  
 3   full_text   9835 non-null   object 
 4   video_id    9835 non-null   object 
 5   public      9835 non-null   bool   
 6   label       9835 non-null   float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 547.5+ KB


In [7]:
# Mengubah tipe data
df['label'] = df['label'].astype('int')
df['label'] = df['label'].astype('category')

<ipython-input-7-888b943be0f9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].astype('int')
<ipython-input-7-888b943be0f9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].astype('category')


In [8]:
# Menghapus data dengan label 3 (komentar judi online)
df = df[df['label'] != 3]

In [9]:
# Mangambil kolom 'full_text' untuk text cleaning
df_text = df[['full_text']]
df_text

,full_text
0,"export di sektor energi hrs dipersulit, krn it..."
1,Pemerintah kenapa ga fokus sama masalah sampah...
2,Belajarlah dari 1MDB Malaysia.
3,Selamat buat DPR punya kebun baru bentar lagi ...
4,Pembahasan lengkap danantara cek Pak Ferry bag...
...,...
9842,Suarax kurang jelas
9843,Dana antara kita saja😂😂
9844,Temasek bagus krn manajemen dan direksi nya ta...
9845,"hajar ajalah selagi orang lain lengah, masalah..."


In [10]:
def remove_URL(text): # Menghapus URL
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_html(text): # Menghapus tag HTML
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

def remove_emoji(text): # Menghapus emoji
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F77F"  # alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251"  # Enclosed characters
                            "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_numbers (text): # Menghapus angka
    text = re.sub(r'\d+', '', text)
    return text

def remove_duplspaces (text): # Menghapus spasi berlebih
    text = re.sub(r'\s+', ' ', text)
    return text

def remove_symbols (text): # Menghapus simbol
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

In [11]:
df_text['cleaning'] = df_text['full_text'].apply(lambda x: remove_URL(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_html(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_emoji(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_symbols(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_numbers(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_duplspaces(x))
df_text

<ipython-input-11-649033b0287e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['cleaning'] = df_text['full_text'].apply(lambda x: remove_URL(x))
<ipython-input-11-649033b0287e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_html(x))
<ipython-input-11-649033b0287e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,full_text,cleaning
0,"export di sektor energi hrs dipersulit, krn it...",export di sektor energi hrs dipersulit krn itu...
1,Pemerintah kenapa ga fokus sama masalah sampah...,Pemerintah kenapa ga fokus sama masalah sampah ya
2,Belajarlah dari 1MDB Malaysia.,Belajarlah dari MDB Malaysia
3,Selamat buat DPR punya kebun baru bentar lagi ...,Selamat buat DPR punya kebun baru bentar lagi ...
4,Pembahasan lengkap danantara cek Pak Ferry bag...,Pembahasan lengkap danantara cek Pak Ferry bag...
...,...,...
9842,Suarax kurang jelas,Suarax kurang jelas
9843,Dana antara kita saja😂😂,Dana antara kita saja
9844,Temasek bagus krn manajemen dan direksi nya ta...,Temasek bagus krn manajemen dan direksi nya ta...
9845,"hajar ajalah selagi orang lain lengah, masalah...",hajar ajalah selagi orang lain lengah masalah ...


In [12]:
# Menghapus duplikasi
df_text = df_text.drop_duplicates(subset='cleaning', keep='first')
df_text

,full_text,cleaning
0,"export di sektor energi hrs dipersulit, krn it...",export di sektor energi hrs dipersulit krn itu...
1,Pemerintah kenapa ga fokus sama masalah sampah...,Pemerintah kenapa ga fokus sama masalah sampah ya
2,Belajarlah dari 1MDB Malaysia.,Belajarlah dari MDB Malaysia
3,Selamat buat DPR punya kebun baru bentar lagi ...,Selamat buat DPR punya kebun baru bentar lagi ...
4,Pembahasan lengkap danantara cek Pak Ferry bag...,Pembahasan lengkap danantara cek Pak Ferry bag...
...,...,...
9842,Suarax kurang jelas,Suarax kurang jelas
9843,Dana antara kita saja😂😂,Dana antara kita saja
9844,Temasek bagus krn manajemen dan direksi nya ta...,Temasek bagus krn manajemen dan direksi nya ta...
9845,"hajar ajalah selagi orang lain lengah, masalah...",hajar ajalah selagi orang lain lengah masalah ...


In [13]:
# Case Folding
def case_folding(text): # Mengubah huruf yang masih uppercase menjadi lowercase
  if isinstance(text, str):
    lowercase_text = text.lower()
    return lowercase_text
  else:
    return text

df_text['case_folding'] = df_text['cleaning'].apply(case_folding)
df_text

<ipython-input-13-41172bf35637>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['case_folding'] = df_text['cleaning'].apply(case_folding)


,full_text,cleaning,case_folding
0,"export di sektor energi hrs dipersulit, krn it...",export di sektor energi hrs dipersulit krn itu...,export di sektor energi hrs dipersulit krn itu...
1,Pemerintah kenapa ga fokus sama masalah sampah...,Pemerintah kenapa ga fokus sama masalah sampah ya,pemerintah kenapa ga fokus sama masalah sampah ya
2,Belajarlah dari 1MDB Malaysia.,Belajarlah dari MDB Malaysia,belajarlah dari mdb malaysia
3,Selamat buat DPR punya kebun baru bentar lagi ...,Selamat buat DPR punya kebun baru bentar lagi ...,selamat buat dpr punya kebun baru bentar lagi ...
4,Pembahasan lengkap danantara cek Pak Ferry bag...,Pembahasan lengkap danantara cek Pak Ferry bag...,pembahasan lengkap danantara cek pak ferry bag...
...,...,...,...
9842,Suarax kurang jelas,Suarax kurang jelas,suarax kurang jelas
9843,Dana antara kita saja😂😂,Dana antara kita saja,dana antara kita saja
9844,Temasek bagus krn manajemen dan direksi nya ta...,Temasek bagus krn manajemen dan direksi nya ta...,temasek bagus krn manajemen dan direksi nya ta...
9845,"hajar ajalah selagi orang lain lengah, masalah...",hajar ajalah selagi orang lain lengah masalah ...,hajar ajalah selagi orang lain lengah masalah ...


## Tokenisasi

In [14]:
def tokenize_with_nltk(text):
    """
    Tokenisasi teks menggunakan NLTK, mengembalikan list kata saja.
    :param text: String teks input
    :return: List token kata
    """
    return word_tokenize(text)

nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [15]:
df_text['token'] = df_text['case_folding'].apply(tokenize_with_nltk)
df_text['token'] = df_text['token'].apply(set)
df_text['token'] = df_text['token'].apply(list)
df_text = df_text.reset_index(drop=True)

<ipython-input-15-d539caa957fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['token'] = df_text['case_folding'].apply(tokenize_with_nltk)
<ipython-input-15-d539caa957fe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['token'] = df_text['token'].apply(set)
<ipython-input-15-d539caa957fe>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [16]:
df_text

,full_text,cleaning,case_folding,token
0,"export di sektor energi hrs dipersulit, krn it...",export di sektor energi hrs dipersulit krn itu...,export di sektor energi hrs dipersulit krn itu...,"[hrs, energi, krn, export, dipersulit, tdk, me..."
1,Pemerintah kenapa ga fokus sama masalah sampah...,Pemerintah kenapa ga fokus sama masalah sampah ya,pemerintah kenapa ga fokus sama masalah sampah ya,"[ya, fokus, ga, sama, masalah, pemerintah, ken..."
2,Belajarlah dari 1MDB Malaysia.,Belajarlah dari MDB Malaysia,belajarlah dari mdb malaysia,"[malaysia, belajarlah, mdb, dari]"
3,Selamat buat DPR punya kebun baru bentar lagi ...,Selamat buat DPR punya kebun baru bentar lagi ...,selamat buat dpr punya kebun baru bentar lagi ...,"[buat, lagi, bentar, kebun, bakal, besaran, ma..."
4,Pembahasan lengkap danantara cek Pak Ferry bag...,Pembahasan lengkap danantara cek Pak Ferry bag...,pembahasan lengkap danantara cek pak ferry bag...,"[pak, mengapa, disalahgunakan, dengan, semua, ..."
...,...,...,...,...
9418,Suarax kurang jelas,Suarax kurang jelas,suarax kurang jelas,"[suarax, jelas, kurang]"
9419,Dana antara kita saja😂😂,Dana antara kita saja,dana antara kita saja,"[antara, kita, saja, dana]"
9420,Temasek bagus krn manajemen dan direksi nya ta...,Temasek bagus krn manajemen dan direksi nya ta...,temasek bagus krn manajemen dan direksi nya ta...,"[semoga, kalo, sprt, tdk, itu, besar, penyalah..."
9421,"hajar ajalah selagi orang lain lengah, masalah...",hajar ajalah selagi orang lain lengah masalah ...,hajar ajalah selagi orang lain lengah masalah ...,"[pembagian, lengah, nanti, ngopi, selagi, bica..."


## Slang

In [17]:
!wget -N -P data/ https://raw.githubusercontent.com/Labibbib/MSA_Case_Study_1_Group-F/refs/heads/main/slang_danantara.txt

--2025-06-05 15:51:20--  https://raw.githubusercontent.com/Labibbib/MSA_Case_Study_1_Group-F/refs/heads/main/slang_danantara.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8258 (8.1K) [text/plain]
Saving to: ‘data/slang_danantara.txt’

slang_danantara.txt 100%[===================>]   8.06K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2025-06-05 15:51:20 (89.5 MB/s) - ‘data/slang_danantara.txt’ saved [8258/8258]



In [18]:
def replace_slang_in_text(file_path, tokens):
    """
    Load the slang dictionary from a file and replace slang words in a list of tokens.
    The file should have lines formatted as "slang:replacement".
    """
    slang_dict = {}
    # Load slang dictionary
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if ':' in line:
                slang, replacement = line.strip().split(':', 1)
                slang_dict[slang.strip()] = replacement.strip()

    # Replace slang words in tokens
    return [slang_dict.get(token, token) for token in tokens]

In [19]:
# Menggunakan file pertama ('slang.txt')
df_text["slang1"] = df_text["token"].apply(lambda tokens: replace_slang_in_text('data/slang_danantara.txt', tokens))

In [20]:
df_text

,full_text,cleaning,case_folding,token,slang1
0,"export di sektor energi hrs dipersulit, krn it...",export di sektor energi hrs dipersulit krn itu...,export di sektor energi hrs dipersulit krn itu...,"[hrs, energi, krn, export, dipersulit, tdk, me...","[harus, energi, karena, export, dipersulit, ti..."
1,Pemerintah kenapa ga fokus sama masalah sampah...,Pemerintah kenapa ga fokus sama masalah sampah ya,pemerintah kenapa ga fokus sama masalah sampah ya,"[ya, fokus, ga, sama, masalah, pemerintah, ken...","[ya, fokus, tidak, sama, masalah, pemerintah, ..."
2,Belajarlah dari 1MDB Malaysia.,Belajarlah dari MDB Malaysia,belajarlah dari mdb malaysia,"[malaysia, belajarlah, mdb, dari]","[malaysia, belajarlah, 1mdb, dari]"
3,Selamat buat DPR punya kebun baru bentar lagi ...,Selamat buat DPR punya kebun baru bentar lagi ...,selamat buat dpr punya kebun baru bentar lagi ...,"[buat, lagi, bentar, kebun, bakal, besaran, ma...","[buat, lagi, sebentar, kebun, bakal, besaran, ..."
4,Pembahasan lengkap danantara cek Pak Ferry bag...,Pembahasan lengkap danantara cek Pak Ferry bag...,pembahasan lengkap danantara cek pak ferry bag...,"[pak, mengapa, disalahgunakan, dengan, semua, ...","[pak, mengapa, disalahgunakan, dengan, semua, ..."
...,...,...,...,...,...
9418,Suarax kurang jelas,Suarax kurang jelas,suarax kurang jelas,"[suarax, jelas, kurang]","[suaranya, jelas, kurang]"
9419,Dana antara kita saja😂😂,Dana antara kita saja,dana antara kita saja,"[antara, kita, saja, dana]","[antara, kita, saja, dana]"
9420,Temasek bagus krn manajemen dan direksi nya ta...,Temasek bagus krn manajemen dan direksi nya ta...,temasek bagus krn manajemen dan direksi nya ta...,"[semoga, kalo, sprt, tdk, itu, besar, penyalah...","[semoga, kalo, seperti, tidak, itu, besar, pen..."
9421,"hajar ajalah selagi orang lain lengah, masalah...",hajar ajalah selagi orang lain lengah masalah ...,hajar ajalah selagi orang lain lengah masalah ...,"[pembagian, lengah, nanti, ngopi, selagi, bica...","[pembagian, lengah, nanti, ngopi, selagi, bica..."


## Stopword

In [21]:
!wget -P data/ https://raw.githubusercontent.com/taudataanalytics/taudata-Academy/refs/heads/main/data/stopwords_id.txt

# Membaca file stopwords_id.txt SEKALI di awal
with open('data/stopwords_id.txt', "r", encoding="utf-8", errors='replace') as file:
    id_stop = file.readlines()

# Memproses dan menghapus duplikat
id_stop = [word.strip().lower() for word in id_stop]
id_stop = list(set(id_stop))

--2025-06-05 15:51:24--  https://raw.githubusercontent.com/taudataanalytics/taudata-Academy/refs/heads/main/data/stopwords_id.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6446 (6.3K) [text/plain]
Saving to: ‘data/stopwords_id.txt’

stopwords_id.txt    100%[===================>]   6.29K  --.-KB/s    in 0s      

2025-06-05 15:51:24 (69.3 MB/s) - ‘data/stopwords_id.txt’ saved [6446/6446]



In [22]:
!wget -N -P data/ https://raw.githubusercontent.com/Labibbib/MSA_Case_Study_1_Group-F/refs/heads/main/stopword_danantara.txt

# Membaca file stopwords_danantara.txt SEKALI di awal
with open('data/stopword_danantara.txt', "r", encoding="utf-8", errors='replace') as file:
    manual_stop = file.readlines()

# Memproses dan menghapus duplikat
manual_stop = [word.strip().lower() for word in manual_stop]
manual_stop = list(set(manual_stop)) # Mengubah ke set untuk menghapus duplikat, lalu kembali ke list

--2025-06-05 15:51:24--  https://raw.githubusercontent.com/Labibbib/MSA_Case_Study_1_Group-F/refs/heads/main/stopword_danantara.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2758 (2.7K) [text/plain]
Saving to: ‘data/stopword_danantara.txt’

stopword_danantara. 100%[===================>]   2.69K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2025-06-05 15:51:24 (37.9 MB/s) - ‘data/stopword_danantara.txt’ saved [2758/2758]



In [23]:
# Fungsi yang menggunakan daftar stopwords yang sudah dimuat
def remove_stopwords(tokens, stopwords_list):
    """
    Remove stopwords from a list of tokens using a predefined stopword list.
    """
    return [token for token in tokens if token not in stopwords_list]

In [24]:
# Terapkan penghapusan stopwords ke kolom 'slang1'
df_text['no_stopwords'] = df_text['slang1'].apply(lambda tokens: remove_stopwords(tokens, id_stop))

# Terapkan penghapusan stopwords ke kolom 'cleaned_text'
df_text['m_no_stopwords'] = df_text['no_stopwords'].apply(lambda tokens: remove_stopwords(tokens, manual_stop))

In [25]:
df_text

,full_text,cleaning,case_folding,token,slang1,no_stopwords,m_no_stopwords
0,"export di sektor energi hrs dipersulit, krn it...",export di sektor energi hrs dipersulit krn itu...,export di sektor energi hrs dipersulit krn itu...,"[hrs, energi, krn, export, dipersulit, tdk, me...","[harus, energi, karena, export, dipersulit, ti...","[energi, export, dipersulit, meningkatkan, pen...","[energi, export, dipersulit, meningkatkan, pen..."
1,Pemerintah kenapa ga fokus sama masalah sampah...,Pemerintah kenapa ga fokus sama masalah sampah ya,pemerintah kenapa ga fokus sama masalah sampah ya,"[ya, fokus, ga, sama, masalah, pemerintah, ken...","[ya, fokus, tidak, sama, masalah, pemerintah, ...","[ya, fokus, pemerintah, sampah]","[fokus, sampah]"
2,Belajarlah dari 1MDB Malaysia.,Belajarlah dari MDB Malaysia,belajarlah dari mdb malaysia,"[malaysia, belajarlah, mdb, dari]","[malaysia, belajarlah, 1mdb, dari]","[malaysia, belajarlah, 1mdb]","[malaysia, belajarlah, 1mdb]"
3,Selamat buat DPR punya kebun baru bentar lagi ...,Selamat buat DPR punya kebun baru bentar lagi ...,selamat buat dpr punya kebun baru bentar lagi ...,"[buat, lagi, bentar, kebun, bakal, besaran, ma...","[buat, lagi, sebentar, kebun, bakal, besaran, ...","[sebentar, kebun, besaran, makan, selamat, dpr]","[sebentar, kebun, besaran, makan, selamat, dpr]"
4,Pembahasan lengkap danantara cek Pak Ferry bag...,Pembahasan lengkap danantara cek Pak Ferry bag...,pembahasan lengkap danantara cek pak ferry bag...,"[pak, mengapa, disalahgunakan, dengan, semua, ...","[pak, mengapa, disalahgunakan, dengan, semua, ...","[disalahgunakan, lembaganya, ferry, value, len...","[disalahgunakan, lembaganya, value, lengkap, b..."
...,...,...,...,...,...,...,...
9418,Suarax kurang jelas,Suarax kurang jelas,suarax kurang jelas,"[suarax, jelas, kurang]","[suaranya, jelas, kurang]",[suaranya],[suaranya]
9419,Dana antara kita saja😂😂,Dana antara kita saja,dana antara kita saja,"[antara, kita, saja, dana]","[antara, kita, saja, dana]",[dana],[]
9420,Temasek bagus krn manajemen dan direksi nya ta...,Temasek bagus krn manajemen dan direksi nya ta...,temasek bagus krn manajemen dan direksi nya ta...,"[semoga, kalo, sprt, tdk, itu, besar, penyalah...","[semoga, kalo, seperti, tidak, itu, besar, pen...","[semoga, kalo, penyalahgunaan, korupsi, danant...","[penyalahgunaan, korupsi, hancurlah, habis, te..."
9421,"hajar ajalah selagi orang lain lengah, masalah...",hajar ajalah selagi orang lain lengah masalah ...,hajar ajalah selagi orang lain lengah masalah ...,"[pembagian, lengah, nanti, ngopi, selagi, bica...","[pembagian, lengah, nanti, ngopi, selagi, bica...","[pembagian, lengah, ngopi, selagi, bicarakan, ...","[pembagian, lengah, ngopi, selagi, bicarakan, ..."


## Lemmatization

In [26]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming menggunakan Sastrawi untuk daftar token
def stem_tokens(tokens):
    """
    Stem each token in a list using Sastrawi stemmer.
    :param tokens: List of tokens to be stemmed.
    :return: List of stemmed tokens.
    """
    return [stemmer.stem(token) for token in tokens]

In [27]:
df_text['lemma_text'] = df_text['m_no_stopwords'].apply(stem_tokens)
df_text

,full_text,cleaning,case_folding,token,slang1,no_stopwords,m_no_stopwords,lemma_text
0,"export di sektor energi hrs dipersulit, krn it...",export di sektor energi hrs dipersulit krn itu...,export di sektor energi hrs dipersulit krn itu...,"[hrs, energi, krn, export, dipersulit, tdk, me...","[harus, energi, karena, export, dipersulit, ti...","[energi, export, dipersulit, meningkatkan, pen...","[energi, export, dipersulit, meningkatkan, pen...","[energi, export, sulit, tingkat, dapat, nguran..."
1,Pemerintah kenapa ga fokus sama masalah sampah...,Pemerintah kenapa ga fokus sama masalah sampah ya,pemerintah kenapa ga fokus sama masalah sampah ya,"[ya, fokus, ga, sama, masalah, pemerintah, ken...","[ya, fokus, tidak, sama, masalah, pemerintah, ...","[ya, fokus, pemerintah, sampah]","[fokus, sampah]","[fokus, sampah]"
2,Belajarlah dari 1MDB Malaysia.,Belajarlah dari MDB Malaysia,belajarlah dari mdb malaysia,"[malaysia, belajarlah, mdb, dari]","[malaysia, belajarlah, 1mdb, dari]","[malaysia, belajarlah, 1mdb]","[malaysia, belajarlah, 1mdb]","[malaysia, bajar, 1mdb]"
3,Selamat buat DPR punya kebun baru bentar lagi ...,Selamat buat DPR punya kebun baru bentar lagi ...,selamat buat dpr punya kebun baru bentar lagi ...,"[buat, lagi, bentar, kebun, bakal, besaran, ma...","[buat, lagi, sebentar, kebun, bakal, besaran, ...","[sebentar, kebun, besaran, makan, selamat, dpr]","[sebentar, kebun, besaran, makan, selamat, dpr]","[sebentar, kebun, besar, makan, selamat, dpr]"
4,Pembahasan lengkap danantara cek Pak Ferry bag...,Pembahasan lengkap danantara cek Pak Ferry bag...,pembahasan lengkap danantara cek pak ferry bag...,"[pak, mengapa, disalahgunakan, dengan, semua, ...","[pak, mengapa, disalahgunakan, dengan, semua, ...","[disalahgunakan, lembaganya, ferry, value, len...","[disalahgunakan, lembaganya, value, lengkap, b...","[disalahgunakan, lembaga, value, lengkap, hubu..."
...,...,...,...,...,...,...,...,...
9418,Suarax kurang jelas,Suarax kurang jelas,suarax kurang jelas,"[suarax, jelas, kurang]","[suaranya, jelas, kurang]",[suaranya],[suaranya],[suara]
9419,Dana antara kita saja😂😂,Dana antara kita saja,dana antara kita saja,"[antara, kita, saja, dana]","[antara, kita, saja, dana]",[dana],[],[]
9420,Temasek bagus krn manajemen dan direksi nya ta...,Temasek bagus krn manajemen dan direksi nya ta...,temasek bagus krn manajemen dan direksi nya ta...,"[semoga, kalo, sprt, tdk, itu, besar, penyalah...","[semoga, kalo, seperti, tidak, itu, besar, pen...","[semoga, kalo, penyalahgunaan, korupsi, danant...","[penyalahgunaan, korupsi, hancurlah, habis, te...","[penyalahgunaan, korupsi, hancur, habis, temas..."
9421,"hajar ajalah selagi orang lain lengah, masalah...",hajar ajalah selagi orang lain lengah masalah ...,hajar ajalah selagi orang lain lengah masalah ...,"[pembagian, lengah, nanti, ngopi, selagi, bica...","[pembagian, lengah, nanti, ngopi, selagi, bica...","[pembagian, lengah, ngopi, selagi, bicarakan, ...","[pembagian, lengah, ngopi, selagi, bicarakan, ...","[bagi, lengah, ngopi, selagi, bicara, hajar, c..."


In [28]:
a = df_text['lemma_text'].to_list()
b = [' '.join(a[i]) for i in range(len(a))]
df_text['cleaned_text'] = b
df_text.head()

,full_text,cleaning,case_folding,token,slang1,no_stopwords,m_no_stopwords,lemma_text,cleaned_text
0,"export di sektor energi hrs dipersulit, krn it...",export di sektor energi hrs dipersulit krn itu...,export di sektor energi hrs dipersulit krn itu...,"[hrs, energi, krn, export, dipersulit, tdk, me...","[harus, energi, karena, export, dipersulit, ti...","[energi, export, dipersulit, meningkatkan, pen...","[energi, export, dipersulit, meningkatkan, pen...","[energi, export, sulit, tingkat, dapat, nguran...",energi export sulit tingkat dapat ngurangin mo...
1,Pemerintah kenapa ga fokus sama masalah sampah...,Pemerintah kenapa ga fokus sama masalah sampah ya,pemerintah kenapa ga fokus sama masalah sampah ya,"[ya, fokus, ga, sama, masalah, pemerintah, ken...","[ya, fokus, tidak, sama, masalah, pemerintah, ...","[ya, fokus, pemerintah, sampah]","[fokus, sampah]","[fokus, sampah]",fokus sampah
2,Belajarlah dari 1MDB Malaysia.,Belajarlah dari MDB Malaysia,belajarlah dari mdb malaysia,"[malaysia, belajarlah, mdb, dari]","[malaysia, belajarlah, 1mdb, dari]","[malaysia, belajarlah, 1mdb]","[malaysia, belajarlah, 1mdb]","[malaysia, bajar, 1mdb]",malaysia bajar 1mdb
3,Selamat buat DPR punya kebun baru bentar lagi ...,Selamat buat DPR punya kebun baru bentar lagi ...,selamat buat dpr punya kebun baru bentar lagi ...,"[buat, lagi, bentar, kebun, bakal, besaran, ma...","[buat, lagi, sebentar, kebun, bakal, besaran, ...","[sebentar, kebun, besaran, makan, selamat, dpr]","[sebentar, kebun, besaran, makan, selamat, dpr]","[sebentar, kebun, besar, makan, selamat, dpr]",sebentar kebun besar makan selamat dpr
4,Pembahasan lengkap danantara cek Pak Ferry bag...,Pembahasan lengkap danantara cek Pak Ferry bag...,pembahasan lengkap danantara cek pak ferry bag...,"[pak, mengapa, disalahgunakan, dengan, semua, ...","[pak, mengapa, disalahgunakan, dengan, semua, ...","[disalahgunakan, lembaganya, ferry, value, len...","[disalahgunakan, lembaganya, value, lengkap, b...","[disalahgunakan, lembaga, value, lengkap, hubu...",disalahgunakan lembaga value lengkap hubung ba...


In [29]:
df = df[df['full_text'].isin(df_text['full_text'])]
df = df.drop_duplicates(subset='full_text', keep='first')

In [30]:
df = df.reset_index(drop=True)

## Memisahkan kolom tanggal dan jam

In [31]:
# Pastikan kolom 'updated_at' adalah datetime
df['updated_at'] = pd.to_datetime(df['updated_at'])

# Memisahkan kolom 'updated_at' menjadi 'date' dan 'time'
# date akan tetap berupa objek datetime.date atau timestamp terlebih dahulu
df['date'] = df['updated_at'].dt.date
df['time'] = df['updated_at'].dt.time

# Mengubah format kolom 'date' menjadi string 'dd/mm/yyyy'
# Konversi ke datetime untuk menggunakan strftime, lalu ke string
df['date'] = pd.to_datetime(df['date']).dt.strftime('%d/%m/%Y')

In [32]:
df_cleaned = df[['label', 'date', 'time', 'author', 'video_id', 'like_count', 'full_text']]
df_cleaned = pd.concat([df_cleaned, df_text['cleaned_text']], axis=1)
df_cleaned.sample(10)

,label,date,time,author,video_id,like_count,full_text,cleaned_text
4192,0,20/02/2025,02:36:13,@aettergaming3274,02_WTfYvQiE,0,"integritas ga beres, malah bikin ginian, ya en...",beres integritas ending gmn cium
719,0,28/02/2025,20:40:47,@bambangadijanto7469,9NRHtpDPv_M,0,"LAHAN BARU UNTUK DI KORUPSI,LEBIH BAIK BANGUN ...",jangka lahan korupsilebih bangun sdm
5969,0,19/02/2025,10:27:34,@mangkubumi1262,02_WTfYvQiE,0,"Liat situasi politiknya bang, kalo carut marut...",bnyak situasi korupsi marut liat carut jabat r...
9102,2,16/02/2025,19:14:10,@fn7004,ruqEjXiJZGw,3,Kita butuh media Seperti ini. Thanks Tempo.,butuh media tempo
4916,1,19/02/2025,16:10:15,@Keepfighting77,02_WTfYvQiE,0,Konten ontollll,ontollll
4318,0,20/02/2025,01:23:04,@Mamajavy,02_WTfYvQiE,0,"Dan si papa dana ternyata mantan koruptor😅, ap...",koruptor mantan percaya
5796,2,19/02/2025,10:55:25,@indraramda030,02_WTfYvQiE,0,bukanya ibu pertiwi harus ngasih makan saudara...,buka makan gratis
3199,0,21/02/2025,05:03:51,@djuraganalnaz2532,02_WTfYvQiE,0,Saya sebagai salah satu pegawai anaknya bu per...,klausul hukum salah investasi investasi temen ...
5044,2,19/02/2025,15:04:13,@Gumilarfams,02_WTfYvQiE,0,Project ini udah pasti atau masih bisa dijegal...,merosot berani gambling proyek jegal tajam mus...
792,0,24/02/2025,11:58:31,@ranijalsitepu4751,9NRHtpDPv_M,0,Sispa Pandu Syahrir? Ada yang tau?,pandu syahrir sispa


## Menambahkan kolom judul video

In [33]:
video_link = []
for video_id in df_cleaned['video_id'].unique():
    link = f'https://www.youtube.com/watch?v={video_id}'
    video_link.append(link)

video_link

['https://www.youtube.com/watch?v=bzJIZc170pE',
 'https://www.youtube.com/watch?v=g3u2XDuQAUw',
 'https://www.youtube.com/watch?v=AS2BXfKHJNU',
 'https://www.youtube.com/watch?v=9NRHtpDPv_M',
 'https://www.youtube.com/watch?v=j5wEDoVdx18',
 'https://www.youtube.com/watch?v=02_WTfYvQiE',
 'https://www.youtube.com/watch?v=WyPLvwLK8c4',
 'https://www.youtube.com/watch?v=PNFTdgS0Mgc',
 'https://www.youtube.com/watch?v=LTrJJ4X3D5Q',
 'https://www.youtube.com/watch?v=ruqEjXiJZGw']

In [34]:
video_title_mapping = {
    'bzJIZc170pE': 'Danantara: Badan Pengelola Investasi yang Olah Aset Rp14.700 Triliun | Narasi Explains',
    'g3u2XDuQAUw': 'Bayang - Bayang Risiko Danantara | Opini Tempo',
    'AS2BXfKHJNU': 'Tempo Eksplainer: Baru Diluncurkan Prabowo, Apa Itu Danantara?',
    '9NRHtpDPv_M': 'Danantara Kelola Aset Hampir Rp15.000 Triliun, Apa Manfaat Bagi Masyarakat Nantinya?',
    'j5wEDoVdx18': 'Risiko TInggi Proyek Danantara | Jelasin Dong!',
    '02_WTfYvQiE': 'Memahami Danantara dengan Mudah',
    'WyPLvwLK8c4': 'Mengenal Apa Itu Danantara',
    'PNFTdgS0Mgc': 'LIVE: Presiden RI Luncurkan Daya Anagata Nusantara (Danantara), 24 Februari 2025',
    'LTrJJ4X3D5Q': 'Bahas Investasi & Danantara Bareng Sandiaga Uno',
    'ruqEjXiJZGw': 'Jalur Cepat RUU BUMN dan Manuver Menguasai Danantara | Jelasin Dong!'
}

channel_mapping = {'bzJIZc170pE':'Narasi Newsroom',
                   'g3u2XDuQAUw':'Tempodotco',
                   'AS2BXfKHJNU':'Tempodotco',
                   '9NRHtpDPv_M':'KOMPASTV',
                   'j5wEDoVdx18':'Tempodotco',
                   '02_WTfYvQiE':'MALAKA',
                   'WyPLvwLK8c4':'CNN Indonesia',
                   'PNFTdgS0Mgc':'Sekretariat Presiden Indonesia',
                   'LTrJJ4X3D5Q':'Timothy Ronald',
                   'ruqEjXiJZGw':'Tempodotco'

}

views_mapping = {
    'bzJIZc170pE':44138,
    'g3u2XDuQAUw':126251,
    'AS2BXfKHJNU':26946,
    '9NRHtpDPv_M':70114,
    'j5wEDoVdx18':207351,
    '02_WTfYvQiE':1950553,
    'WyPLvwLK8c4':115070,
    'PNFTdgS0Mgc':176458,
    'LTrJJ4X3D5Q':937500,
    'ruqEjXiJZGw':824926
}

In [35]:
df_cleaned['judul_video'] = df_cleaned['video_id'].map(video_title_mapping)
df_cleaned['channel_yt'] = df_cleaned['video_id'].map(channel_mapping)
df_cleaned['views_count'] = df_cleaned['video_id'].map(views_mapping)

In [36]:
df_cleaned.sample(10)

,label,date,time,author,video_id,like_count,full_text,cleaned_text,judul_video,channel_yt,views_count
303,0,25/02/2025,08:49:28,@raineeflame-14all41,g3u2XDuQAUw,0,"Konsep nya keren , karakter dalam management t...",korporasi kaya telunjuk hukum laku koruptor ma...,Bayang - Bayang Risiko Danantara | Opini Tempo,Tempodotco,126251
4391,0,20/02/2025,00:50:47,@BregasAdji,02_WTfYvQiE,0,Ya. Apalagi yang menjabat di danantara katanya...,tempat kocak jokowi didanantara jabat kendali ...,Memahami Danantara dengan Mudah,MALAKA,1950553
8808,2,23/03/2025,11:54:05,@toplesastor7818,LTrJJ4X3D5Q,1,gasss,gasss,Bahas Investasi & Danantara Bareng Sandiaga Uno,Timothy Ronald,937500
9365,1,15/02/2025,08:31:35,@goesmadi550,ruqEjXiJZGw,1,Apa yang mau di harapkan dari presiden yang di...,dmn referensi shincan baca mentor olehmulyono ...,Jalur Cepat RUU BUMN dan Manuver Menguasai Dan...,Tempodotco,824926
2371,0,24/02/2025,08:21:55,@royofpalar9912,02_WTfYvQiE,0,"Resikonya lebih besar, apalagi jika ada orang ...",korupsi dalam mantan napi resiko,Memahami Danantara dengan Mudah,MALAKA,1950553
1891,0,25/02/2025,20:57:46,@NurHidayat-f6c,02_WTfYvQiE,0,"Bang, lu tau ga kalo direktur Pertamina melaku...",direktur korupsi besar pertamina,Memahami Danantara dengan Mudah,MALAKA,1950553
1418,0,28/02/2025,04:47:17,@theodorus_es,02_WTfYvQiE,0,Baru sadar kalau watermark close the door suda...,buzzer sadar pikir wkwkk kongsi ubah watermark...,Memahami Danantara dengan Mudah,MALAKA,1950553
5608,0,19/02/2025,11:34:05,@orenjida4041,02_WTfYvQiE,0,pemerintah bikin lembaga kaya danantara ini tu...,publik kacau percaya turun beban kaya mbg hade...,Memahami Danantara dengan Mudah,MALAKA,1950553
7694,0,26/03/2025,05:09:12,@asgard-hehe,LTrJJ4X3D5Q,0,"pak sandi sedang ngobrol sama devil, begitu ki...",sandi devil kejut ngobrol,Bahas Investasi & Danantara Bareng Sandiaga Uno,Timothy Ronald,937500
4900,2,19/02/2025,16:21:28,@WhatTheHealthBuddy,02_WTfYvQiE,0,"gua cinta banget sama lu bang, semoga lu selal...",edukasi lindung abadi semangat niat buruk cinta,Memahami Danantara dengan Mudah,MALAKA,1950553


## Membuat dataframe untuk analisis BERTopic

In [40]:
def stopword_berd(text):
    indonesian_stopwords = stopwords.words('indonesian')
    text = " ".join([word for word in text.split() if word not in indonesian_stopwords])
    return text

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [38]:
df_brt = df[['full_text']]

In [41]:
df_brt['clean_text'] = df_brt['full_text'].apply(lambda x: remove_URL(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_html(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_symbols(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_numbers(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_duplspaces(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: case_folding(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: stopword_berd(x))
df_brt

<ipython-input-41-b22d9db96b83>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brt['clean_text'] = df_brt['full_text'].apply(lambda x: remove_URL(x))
<ipython-input-41-b22d9db96b83>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_html(x))
<ipython-input-41-b22d9db96b83>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

,full_text,clean_text
0,"export di sektor energi hrs dipersulit, krn it...",export sektor energi hrs dipersulit krn tdk me...
1,Pemerintah kenapa ga fokus sama masalah sampah...,pemerintah ga fokus sampah ya
2,Belajarlah dari 1MDB Malaysia.,belajarlah mdb malaysia
3,Selamat buat DPR punya kebun baru bentar lagi ...,selamat dpr kebun bentar besaran makan
4,Pembahasan lengkap danantara cek Pak Ferry bag...,pembahasan lengkap danantara cek ferry bagus p...
...,...,...
9418,Suarax kurang jelas,suarax
9419,Dana antara kita saja😂😂,dana
9420,Temasek bagus krn manajemen dan direksi nya ta...,temasek bagus krn manajemen direksi nya takut ...
9421,"hajar ajalah selagi orang lain lengah, masalah...",hajar ajalah selagi orang lengah pembagian bic...


# Menyimpan dataset

In [42]:
df_cleaned.to_excel('df_cleaned.xlsx', index=False)
df_brt.to_excel('df_brt.xlsx', index=False)